# POLISCI 3

## Week 10 Lecture Notebook 2: Bivariate regression

Welcome to Week 10, Lecture 2!

In the last lecture, we saw how useful it is to be able to put a "line of best fit" on a graph. It really helps get a sense of trends in the data.

Today we're going to talk about how those lines of best fit are calculated and how we can use R to retrieve the formulas for them. The approach we use for this is called _bivariate regression_. The "bivariate" part is because we're just using two variables: a y-variable (vertical axis) and an x-variable (horizontal axis).

How is this line calculated? This is a bit technical and I don't need you to remember this _exactly_, but I want to understand the basics: A bivariate regression line is formed by finding the line that minimizes the square of the vertical distances between the line and each of the points.

I think this is easiest to see visually, by thinking of the best fit line as suspended from springs. Here's what this would look like for some of the data from Week 2:

![](regressionexample.png)

One of the things this makes clear is that one data point way off from the others can have a big impact on the line of best fit. We call this an _outlier_.

![](withoutlier.png)

### Getting the formula for the line of best fit

Remember algebra? You can describe a line by $y = mx + b$. That is:

$\text{Variable of Interest (y)} = \text{Slope of line of best fit (m)} * \text{Explanatory variable (x)} + \text{Y-Intercept of line of best fit (b)}$

The y-intercept is at what value of y the line crosses the y-axis; i.e., the value of the line when x is zero.

The slope is how much the line "rises" for every unit of increase in x.

<img src="ymxb.jpg">

So, this picture has a slope of 0.5 and an intercept of 1. The formula for this line is therefore $y = 0.5 * x + 1$.

Notice that we can use this formula to figure out where the line is (that is, what the value of y will be) for any value of x we give it. For example, where is the line when x = 5? $0.5 * 5 + 1 = 3.5$. And sure enough, that is what we see on the graph.

### Getting the formula for the line of best fit in R

To get the formula for line of best fit of a dataset in R, we run this command:

```
summary(lm(y.variable ~ x.variable, data = data))
```

A little terminology:
- For the `y.variable`: We call this variable either "outcome variables" or "dependent variables" because they are the outcome we are trying to predict (or, if we have a causal interpretation, that we think might _depend_) on the other variable. Sometimes we also say "left hand side" variable.
- For the `x.variable`: We call this variable either "predictor variables," "independent variables," or "right hand side variables."

Let's now run our first bivariate regression.

### Data example

In [ ]:
library(ggplot2)

data <- read.csv('broockman_skovron.csv')
head(data)

Here is an overview of what the columns in the dataset represent: 

- `republican`: whether the state legislator or state legislative candidate who answered the survey was a Republican (1 = Republican, 0 = Democrat)
- `incumbent`: whether the person who answered the survey was an incumbent representative (i.e., currently holds political office) (1 = already holds office, 0 = just a candidate)
- `marriage_perception`: what percent of the politician's constituents they think support same-sex marriage
- `marriage_opinion`: what percent of the politician's constituents actually support same-sex marriage
- `religexempt_perception`: what percent of the politician's constituents they think support allowing employers to refuse to cover birth control for employees if it violates their religious beliefs
- `religexempt_opinion`: what percent of the politician's constituents actually support allowing employers to refuse to cover birth control for employees if it violates their religious beliefs
- `gunsbanassault_perception`: what percent of the politician's constituents they think support banning assault weapons
- `gunsbanassault_opinion`: what percent of the politician's constituents actually support banning assault weapons

## Bivariate Regression

First, as a reminder, here's what the variables we are working with look like:

In [ ]:
qplot(marriage_opinion, marriage_perception, data = data) + geom_smooth(method = 'lm') + xlim(0,100)

Now let's run a regression.

(Notice: the order for `qplot()` and `summary(lm())` are opposite!)

In this example, `marriage_perception` is our outcome/left hand side/dependent/y axis variable, and `marriage_opinion` is our predictor/right hand side/independent/x axis variable.

In [ ]:
summary(lm(marriage_perception ~ marriage_opinion, data))

Let's interpret each of these numbers:

- `(Intercept)` is showing us the **y-intercept** of the line of best fit. Literally interpreted, it is the line of best fit's best guess for what the value of y would be when x is zero. This doesn't necessarily need to make sense. In this case, there isn't any data at zero, and so we shouldn't literally predict that in a district where 0% of people supported same-sex marraige (the x variable) that the average politician would say -22% of people support it --- that number doesn't even make sense. It just means that, if we kept following the line of best fit, it would eventually hit the y axis at around -22.
- `marriage_opinion` shows the slope of the line of best fit. We also call this the "coefficient for `marriage_opinion`" because it shows us the $m$ that is multiplied by the $x$ (here, `marriage_opinion`).

We can do something pretty useful with these numbers! Say we want to form a best guess for what percent of politicians think their constituents support gay marraige in a district where 50% of voters actually do support gay marriage. That is, we want to know the $y$ in $y = mx + b$. We have $m$, $x$, and $b$, so we can do it!

- $b$ is the estimate for the `(Intercept)` term (-21.86761)
- $m$ is the estimate for the coefficeint on `marriage_opinion` (1.21777)
- $x$ we've set at 50%; this is the kind of district we want to know about

Let's plug this in:

In [ ]:
best.guess.at.50 <- 1.21777 * 50 + -21.86761
best.guess.at.50

### Don't extrapolate beyond the data

Now we have a formula where we can take any value of $x$ (here, `marriage_opinion`) and get a best guess outcome/$y$ (here, `marriage_perception`). That is pretty cool!

But we need to be careful when we do this. The line of best fit can give us a best guess for _any_ value of $x$ we give it---it's just a formula. But some values of $x$ don't make sense to use because there is no data there!

Let's take a look at the graph again.

In [ ]:
qplot(marriage_opinion, marriage_perception, data = data) + geom_smooth(method = 'lm') + xlim(0,100)

The district in the entire country that is estimated to be most conservative is estimated to have 32% of its people support same-sex marriage:

In [ ]:
min(data$marriage_opinion, na.rm = T)

So, we *could* compute our best guess of how legislators would percieve a hypothetical district at 10% support:

In [ ]:
-21.86 + 1.21 * 10

...but **this would be a bad idea**. There isn't any data there, and so we're looking past the domain of the data. Marriage opinion is never below 32, so we shouldn't extrapolate to values where data doesn't exist; our answers could be non-sensical.

For example, the `(Intercept)` is telling us what the regression's best guess is when x = 0 --- i.e., in a district where no voters supported same-sex marriage, what would a politician percieve support as? There's no data here, though, and so the answer we get from the regression is non-sensical (the intercept is at -22%).

This is part of why it's so important to plot our data visually when we want to make predictions: we can make sure we aren't extrapolating outside of where we have data.

### A note on "correlation"

You've probably heard the phrase "correlation." In addition to being used in a more general way to mean "a relationship between two variables," there is actually a technical definition of a correlation. A correlation is a metric used to how well you can predict each variable with the other.

There's really only three things I want you to know about correlations:
- Correlations range from -1 to 1.
- A correlation of 0 means there is no relationship between two variables. The slope of the line of best fit is therefore 0.
- A correlation of 1 means you can perfectly predict one variable from the other (and -1 means the same, except one is the opposite of the other; they are perfectly negatively related).

So, if someone mentions a correlation, just think, "they are telling me how well I can predict these variables with each other."

But the correlation measure has some major problems. In fact, social scientists don't use correlations very much any more, and so I've decided not to spend much time on it in this class.

I want to briefly explain why correlations are problematic for those who are interested. If you don't care about correlations, you don't need to watch this part and you can stop watching this video now. I'm not going to ask you about correlations in class since they are not used very often any more. But for those of you who are interested, I am going to explain why they have problems.

The big problem with correlations is that they measure _how well we can predict the outcome with the prediction variable_, but they **don't** measure the _strength_ of the relationship between the two.

I can show you this with an example.

You can compute correlations right from the bivariate regression output -- it is the square root of the value called the `R-squared` you see below:

In [ ]:
summary(lm(marriage_perception ~ marriage_opinion, data))

In [ ]:
sqrt(0.2813)

In [ ]:
cor(data$marriage_perception, data$marriage_opinion, use = 'complete')

So, this is what a correlation of around 0.53 looks like:

In [ ]:
qplot(marriage_opinion, marriage_perception, data = data) + geom_smooth(method = 'lm')

Let's add some noise to this data:

In [ ]:
data$marriage_perception <- data$marriage_perception + rnorm(nrow(data), sd = 20) 

After adding some noise, the data looks like this:

In [ ]:
qplot(marriage_opinion, marriage_perception, data = data) + geom_smooth(method = 'lm')

Notice, though, that the line of best fit is about the same. We can verify this with a regression.

In [ ]:
summary(lm(marriage_perception ~ marriage_opinion, data))

But, the correlation is now lower!

In [ ]:
sqrt(0.1594)

This example shows the problem with correlations. Before and after I added some noise to the y variable, the line of best fit was the same. For a social scientist, we still make the same predictions --- and, more to the point, the slope of the line is basically the same, so the strength of the relationship is the same!

But if we looked at the correlation, we would think the second relationship is weaker. It's not. It's just that it got harder to predict the outcome because there was extra noise in it. Correlations therefore measure _predictability_ more than they do _strength of a relationship_, and this is why social scientists tend not to use them.